In [2]:
import sys
import os

sys.path.append('/gpfs/helios/home/ploter/projects/MultiSensorDropout/')

In [3]:
import argparse
import torch

args = argparse.Namespace(
    # Basic training parameters
    seed=42,
    batch_size=1,
    epochs=18,
    learning_rate=1e-3,
    learning_rate_backbone=1e-4,
    learning_rate_backbone_names=["backbone"],
    weight_decay=0.01,
    scheduler_step_size=12,
    eval_interval=1,
    patience=5,
    model='perceiver',
    backbone='cnn',  # Set specific value for evaluation
    eval=True,
    weight_loss_center_point=5,
    weight_loss_bce=1,
    shuffle_views=False,
    object_detection=True,  # Set to True for object detection
    
    # Matcher parameters
    set_cost_class=1,
    set_cost_bbox=5,
    set_cost_giou=2,
    focal_alpha=0.25,
    focal_gamma=2,
    
    # Loss coefficients
    bbox_loss_coef=5,
    giou_loss_coef=2,
    eos_coef=0.1,
    
    # Checkpoint parameters
    resume='checkpoint_epoch_17.pth',  # Replace with actual checkpoint path
    output_dir="./output",  # Provide a default output directory
    device='cuda' if torch.cuda.is_available() else 'cpu',
    
    # Dataset parameters
    dataset='moving-mnist',
    dataset_path='Max-Ploter/detection-moving-mnist-easy',
    generate_dataset_runtime=False,
    num_workers=4,
    num_frames=20,
    train_dataset_fraction=1.0,
    train_dataset_size=1.0,  # Additional parameter from notebook
    test_dataset_fraction=1.0,
    frame_dropout_pattern=None,
    view_dropout_probs=[],#[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85],
    sampler_steps=[], #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    sequential_sampler=False,
    grid_size=(1, 1),
    tile_overlap=0.0,
    
    # Wandb parameters
    wandb_project='multi-sensor-dropout',
    wandb_id=None,
    
    # Perceiver model parameters
    num_freq_bands=6,  # Set to 4 as in the notebook
    max_freq=10, 
    enc_layers=1,
    num_queries=16,  # Set to 16 as in the notebook
    hidden_dim=128,
    enc_nheads_cross=1,
    nheads=1,
    dropout=0.0,
    self_per_cross_attn=1,
    multi_classification_heads=False,
    
    # LSTM model parameters
    lstm_hidden_size=128,
    
    # Additional parameters for complete compatibility
    focal_loss=True,
)

In [4]:
class DummyCriterion:
    def __init__(self, device):
        # Initialize with the necessary attributes
        self.device = device
        self.weight_dict = {'loss_ce': 1}
        
    def __call__(self, outputs, targets, *args, **kwargs):
        # Return a dictionary with zero losses to maintain the expected interface
        return {
            'loss_ce': torch.tensor(0.0, device=self.device),
            'loss_bbox': torch.tensor(0.0, device=self.device),
            'loss_giou': torch.tensor(0.0, device=self.device),
            'loss': torch.tensor(0.0, device=self.device)
        }

In [24]:
import torch
from torch import nn

class PostProcessTopK(nn.Module):
    """ Wrapper that applies a post-processor and keeps only the top-k predictions by score and a score threshold """

    def __init__(self, post_processor, top_k=10, score_threshold=0.5):
        super().__init__()
        self.post_processor = post_processor
        self.top_k = top_k
        self.score_threshold = score_threshold

    @torch.no_grad()
    def forward(self, outputs, target_sizes):
        # Get results from the original post processor
        results = self.post_processor(outputs, target_sizes)

        # Filter to keep only top-k results and results above the score threshold
        filtered_results = []
        for result in results:
            scores, labels, boxes = result['scores'], result['labels'], result['boxes']

            # Get top-k indices by score
            top_k = min(self.top_k, len(scores))
            if top_k > 0:  # Check if there are any predictions
                top_indices = torch.topk(scores, top_k).indices

                # Filter by top-k indices
                top_scores = scores[top_indices]
                top_labels = labels[top_indices]
                top_boxes = boxes[top_indices]

                # Filter by score threshold
                threshold_indices = top_scores >= self.score_threshold
                final_scores = top_scores[threshold_indices]
                final_labels = top_labels[threshold_indices]
                final_boxes = top_boxes[threshold_indices]

                filtered_results.append({'scores': final_scores, 'labels': final_labels, 'boxes': final_boxes})
            else:
                # Keep empty result if no predictions
                filtered_results.append({'scores': torch.empty(0), 'labels': torch.empty(0, dtype=torch.int64), 'boxes': torch.empty(0, 4)})

        return filtered_results

In [25]:
from engine import evaluate
from dataset import build_dataset
import torch
from engine import evaluate
from dataset import build_dataset
from models import build_model
from models.perceiver import PostProcess
import os
from pathlib import Path
from util.misc import collate_fn, is_main_process, get_sha, get_rank
from torch.utils.data import DataLoader


args.resume = 'checkpoint_epoch_17.pth'
args.output_dir = "../not_tracked_dir/output_perceiver_detection_2025-04-14_09-56-40"
args.test_dataset_fraction = 1

checkpoint_path = os.path.join(args.output_dir, args.resume)

dataset_test = build_dataset(split='test', args=args)

sampler_test = torch.utils.data.SequentialSampler(dataset_test)

dataloader_test = DataLoader(dataset_test, sampler=sampler_test, batch_size=args.batch_size,
                                collate_fn=collate_fn, num_workers=args.num_workers, pin_memory=True)

model = build_model(args, dataset_test.input_image_view_size)

device = torch.device(args.device)
postprocessors = {
    'bbox': PostProcessTopK(PostProcess())
}

criterion = DummyCriterion(device)

if os.path.exists(checkpoint_path):
	print(f"Loading checkpoint from {checkpoint_path}")
	checkpoint = torch.load(checkpoint_path, map_location=args.device)
	model.load_state_dict(checkpoint['model'])
else:
	raise FileNotFoundError(f"Checkpoint not found at {checkpoint_path}")

model.to(args.device)

results = evaluate(
	model=model,
	dataloader=dataloader_test,
	criterion=criterion,  # Not needed for evaluation
	postprocessors=postprocessors,
	epoch=-1,
	device=device,
)

print("Evaluation Results:")
for metric_name, value in results.items():
	print(f"{metric_name}: {value:.4f}")

Generating test huggingface MovingMNIST dataset...


/tmp/ipykernel_3882979/1649666546.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=args.device)


Using object detection mode
Transforms: Compose(
)
num_freq_bands: 6
depth: 1
max_freq: 10
input_channels: 32
input_axis: 2
num_latents: 16
latent_dim: 128
cross_heads: 1
latent_heads: 1
cross_dim_head: 58
latent_dim_head: 128
num_classes: -1
attn_dropout: 0.0
ff_dropout: 0.0
weight_tie_layers: False
fourier_encode_data: True
self_per_cross_attn: 1
final_classifier_head: False
num_sensors: 1
__class__: <class 'models.perceiver.Perceiver'>
Loading checkpoint from ../not_tracked_dir/output_perceiver_detection_2025-04-14_09-56-40/checkpoint_epoch_17.pth


Eval -1:: 100%|██████████| 10000/10000 [14:22<00:00, 11.59it/s, loss_running=0, class_error_running=nan, loss_center_point_running=nan, loss_ce_running=0, loss=0, loss_ce_unscaled=0, loss_bbox_unscaled=0, loss_giou_unscaled=0, loss_unscaled=0, loss_ce=0, view_dropout_prob=-1]



Raw mAP Results: {'map': tensor(0.6828), 'map_50': tensor(0.8640), 'map_75': tensor(0.8098), 'map_small': tensor(0.6828), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(0.5995), 'mar_10': tensor(0.7301), 'mar_100': tensor(0.7301), 'mar_small': tensor(0.7301), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32)}

Evaluation Results:
loss_running: 0.0000
class_error_running: nan
loss_center_point_running: nan
loss_ce_running: 0.0000
loss: 0.0000
loss_ce_unscaled: 0.0000
loss_bbox_unscaled: 0.0000
loss_giou_unscaled: 0.0000
loss_unscaled: 0.0000
loss_ce: 0.0000
mAP_map: 0.6828
mAP_map_50: 0.8640
mAP_map_75: 0.8098
mAP_map_small: 0.6828
mAP_map_medium: -1.0000
mAP_map_large: -1.0000
mAP_mar_1: 0.5995
mAP_mar_10: 0.7301
mAP_mar_100: 0.7301
mAP_mar_small: 0.7301
mAP_mar_medium: -1.0000
mAP_mar_large: -1.0000
mAP_map_per_class: -1.0000
mA

## RESULT

Raw mAP Results: {'map': tensor(0.7331), 'map_50': tensor(0.9379), 'map_75': tensor(0.8674), 'map_small': tensor(0.7331), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(0.6338), 'mar_10': tensor(0.7885), 'mar_100': tensor(0.7885), 'mar_small': tensor(0.7885), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32)}

Evaluation Results:
loss_running: 0.0000
class_error_running: nan
loss_center_point_running: nan
loss_ce_running: 0.0000
loss: 0.0000
loss_ce_unscaled: 0.0000
loss_bbox_unscaled: 0.0000
loss_giou_unscaled: 0.0000
loss_unscaled: 0.0000
loss_ce: 0.0000
mAP_map: 0.7331
mAP_map_50: 0.9379
mAP_map_75: 0.8674
mAP_map_small: 0.7331
mAP_map_medium: -1.0000
mAP_map_large: -1.0000
mAP_mar_1: 0.6338
mAP_mar_10: 0.7885
mAP_mar_100: 0.7885
mAP_mar_small: 0.7885
mAP_mar_medium: -1.0000
mAP_mar_large: -1.0000
mAP_map_per_class: -1.0000
mAP_mar_100_per_class: -1.0000

## Result
top 10 and threashold 0.5

Raw mAP Results: {'map': tensor(0.6828), 'map_50': tensor(0.8640), 'map_75': tensor(0.8098), 'map_small': tensor(0.6828), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(0.5995), 'mar_10': tensor(0.7301), 'mar_100': tensor(0.7301), 'mar_small': tensor(0.7301), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32)}

Evaluation Results:
loss_running: 0.0000
class_error_running: nan
loss_center_point_running: nan
loss_ce_running: 0.0000
loss: 0.0000
loss_ce_unscaled: 0.0000
loss_bbox_unscaled: 0.0000
loss_giou_unscaled: 0.0000
loss_unscaled: 0.0000
loss_ce: 0.0000
mAP_map: 0.6828
mAP_map_50: 0.8640
mAP_map_75: 0.8098
mAP_map_small: 0.6828
mAP_map_medium: -1.0000
mAP_map_large: -1.0000
mAP_mar_1: 0.5995
mAP_mar_10: 0.7301
mAP_mar_100: 0.7301
mAP_mar_small: 0.7301
mAP_mar_medium: -1.0000
mAP_mar_large: -1.0000
mAP_map_per_class: -1.0000
mAP_mar_100_per_class: -1.0000